# 发布到云原生环境

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 本章代码

</div>

* [https://github.com/niuyuping/Cloud-Native-Full-Stack-Engineering-Course/tree/main/Chapter04/Code](https://github.com/niuyuping/Cloud-Native-Full-Stack-Engineering-Course/tree/main/Chapter04/Code)

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 演示地址

</div>

* [https://social-insurance-frontend-chapter03-811681750681.asia-northeast1.run.app](https://social-insurance-frontend-chapter03-811681750681.asia-northeast1.run.app)

<style>
    .terminal-span {
        margin: 0 0.1rem !important; 
        padding: 0.1rem 0.2rem !important; 
        background-color: #d3e3fd !important;
        color: black !important;
        border-radius: 0.25rem !important;
    }
    .terminal-div {
        padding: 1rem; 
        background-color: #d3e3fd; 
        color: black; 
        width: 40rem; 
        margin-bottom: 1rem; 
        border-radius: 0.25rem; 
        margin-left: 1rem; 
    }
</style>

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 目标

</div>  

* 将前面章节中构建的前端和后端服务发布到<span class="terminal-span">Google Cloud Run</span>上，使用云数据库<span class="terminal-span">Cloud SQL</span>来代替原生PostgreSQL数据库。
 
    <div style="margin-bottom: 1rem"></div>

* #### Github -> Cloud Run持续发布

    <img src="../../Assets/github-build-delploy.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>

* #### 扩展知识
  
    * Cloud Run 是在 Google Cloud 上部署无状态应用的服务，它结合了 Kubernetes 的灵活性和 Serverless 的便捷性。如果你有一个可以通 过 HTTP 访问的 Web 应用或 API，且不需要持久化存储在本地磁盘，那么 Cloud Run 以及同类服务通常是性价比最高且最省心的选择。
    
    * Google Cloud SQL 是一项全托管关系型数据库服务，它在云端提供 MySQL、PostgreSQL 或 SQL Server 数据库，不需要自己去购买服务器、安装数据库软件、处理备份或打补丁，是传统 Web 应用迁移上云或新开发应用最常用的数据库选择。
    
     <div style="margin-bottom: 1rem"></div>

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 技术栈

</div>

* #### 基础设施

    * Docker - [https://www.docker.com/](https://www.docker.com/)

    * Google Cloud - [https://cloud.google.com](https://cloud.google.com) 

    <div style="margin-bottom: 1rem"></div>


<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 系统架构图

</div>  

<img src="../../Assets/Deploy_to_Cloud_Run_01.jpg" alt="Social Insurance Query System" style="max-width: 100%; margin-left: 25px; padding: 50px; background-color: white">

<style>
    .terminal-span {
        margin: 0 0.1rem !important; 
        padding: 0.1rem 0.2rem !important; 
        background-color: #d3e3fd !important;
        color: black !important;
        border-radius: 0.25rem !important;
    }
    .terminal-div {
        padding: 1rem; 
        background-color: #d3e3fd; 
        color: black; 
        width: 40rem; 
        margin-bottom: 1rem; 
        border-radius: 0.25rem; 
        margin-left: 1rem; 
    }
</style>

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 创建数据库

</div>  

* #### 创建Cloud SQL PostgreSQL实例

    * ① 导航到[Cloud Run开始页面](https://console.cloud.google.com/sql/get-started)。
  
    * ② 依次点击「创建实例 -> 选择PostgreSQL」 按钮。
  
    * ③ 在Cloud SQL页面中依次配置以下参数: 

        * Cloud SQL版本 -> 企业版（重要）

        * 预设值 ->  沙盒

        * 实例信息
        
            * 数据库版本 -> PostgreSQL 17

            * 实例ID -> Social-Insurance-Database

            * 密码 -> {安全的密码}

        * 区域 -> {离你的用户最近的区域}

        * 单个可用区

        * 自定义实例

            * 机器配置 -> 通用 - 共享核心 -> 1vCPU, 0.614 GB

            * 存储空间 -> HDD -> 10 GB
        
        * 连接数

            * 实例IP分配 -> 专用IP（选中）

            * 选择专用连接方法 -> 专用服务访问通道(PSA)（选中） -> （用默认配置创建一个VPC网络）

            * 公共IP（不选中）
     
    * ④ 点击「保存」按钮。
    
    <div style="margin-bottom: 1rem"></div>

* #### 创建数据库
  
    * ① 导航到[Cloud Run实例页面](https://console.cloud.google.com/sql/instances)。
  
    * ② 依次点击「实例 -> Social-Insurance-Database」

    * ③ 依次点击「数据库 -> 创建数据库」

        * 数据库名称 -> social_insurance

        * 点击「创建」按钮
  
    <div style="margin-bottom: 1rem"></div>

* #### 创建数据库用户

    * ① 导航到[Cloud Run实例页面](https://console.cloud.google.com/sql/instances)。
  
    * ② 依次点击「实例 -> Social-Insurance-Database」

    * ③ 依次点击「用户 -> 添加用户帐号」

        * 内置身份验证（选中）

        * 用户名 -> db_user

        * 密码 -> {输入一个安全的密码}

        * 点击「添加」按钮

    <div style="margin-bottom: 1rem"></div>

* #### 配置内部专用网络
  
    * ① 导航到[Cloud Run实例页面](https://console.cloud.google.com/sql/instances)。
  
    * ② 依次点击「实例 -> Social-Insurance-Database」

    * ③ 依次点击「连接 -> 摘要」

    * ④ 记下「专用IP连接 -> 服务连接方法 -> 内部IP地址」，在部署后端服务时会用到这个IP地址。
  
    <div style="margin-bottom: 1rem"></div>

* #### 扩展知识
  
    * 我们本章建立的Cloud SQL实例是按照最经济的标准创建的，意味着性能很低，花费也很低，主要用于学习目的。如果是生产环境，则需要按照实际需求重新配置。

    * VPC是虚拟私有云（Virtual Private Cloud）的缩写，它构建在Google Cloud、AWS这类公有云上，VPC网络是在VPC内部建立的专用网络，数据和流量在逻辑上与其他用户的流量完全分开，互不干扰，我们可以在VPC网络上定义独立路由和安全策略。VPC也可以通过虚拟专用网络（VPN）或专线与本地私有云（Private Cloud）连接，构建混合云（Hybrid Cloud）。


<style>
    .terminal-span {
        margin: 0 0.1rem !important; 
        padding: 0.1rem 0.2rem !important; 
        background-color: #d3e3fd !important;
        color: black !important;
        border-radius: 0.25rem !important;
    }
    .terminal-div {
        padding: 1rem; 
        background-color: #d3e3fd; 
        color: black; 
        width: 40rem; 
        margin-bottom: 1rem; 
        border-radius: 0.25rem; 
        margin-left: 1rem; 
    }
</style>

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 发布后端服务

</div>

* #### 添加同源策略

    <img src="../../Assets/react-dom.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>

* #### 创建Dockerfile

    <img src="../../Assets/react-dom.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>

* #### 创建生产环境配置文件
  
  * ① 在Cursor首页按下<span class="terminal-span">Shift + Command + P</span>(MacOS)或<span class="terminal-span">Shift + Ctrl + P </span>(Windows)

  * ② 输入 _<span style="color: orange">git: clone</span>_

  * ③ 输入<span class="terminal-span">第1章中的原型代码库</span>地址（Github代码库URL）。

  * ④ 打开克隆下来的代码。 

    <div style="margin-bottom: 1rem"></div>

* #### 提交并推送代码到Github

    <img src="../../Assets/tailwind-css-fix.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>

* #### 创建Cloud Run服务

  * 在Cursor的Chat(Agent模式)中输入: 
  
    * <span style="color: orange">
    
      _我的项目原型是在Google AI Studio上构建的，现在在本地构建时需要将TailWind以及其它依赖库的导入方式从运行时模式变更为构建时模式，请分析项目中的所有代码，并帮我完成修改。_

    </span>  

  <div style="margin-bottom: 1rem"></div>

* #### 使用Httpie进行测试

  * 在Cursor的<span class="terminal-span">终端</span>窗口里输入并按下回车:   

    * _<span style="color: orange">
          http {Cloud Run App Url}/socialInsuranceQuery monthlySalary==300000 age==45
      </span>_

    <div style="margin-bottom: 1rem; margin-left: 1rem; padding-top: 1rem">

    <div style="margin-bottom: 1rem"></div>

* #### 扩展知识
      
    * <span class="terminal-span">面向对象编程</span>（OOP）是最重要的编程思想和范式之一，类（Class）、对象（Object）、继承（Inherited）、多态（Polymorphism）、方法（Method）都是它的核心概念，深入理解面向对象的软件开发方法，是对一名软件工程师的基本要求。

    * <span class="terminal-span">Axios</span> 是一个基于 <span class="terminal-span">Promise</span> 的 HTTP 客户端，用于在浏览器和 Node.js 中发送网络请求。它能够自动转换 JSON 数据，提供更好的错误处理机制，同时还实现了<span class="terminal-span">拦截器</span>，这些特性极大提高了前端开发的效率。

    * <span class="terminal-span">DevTools</span>（Developer Tools）是直接内置在现代网络浏览器（如 Chrome, Firefox, Edge, Safari）中的一套强大的 Web 开发和调试工具集。在浏览器中按下<span class="terminal-span">Fn+F12</span>(MacOS)或<span class="terminal-span">F12</span>(Windows)即可调出DevTools的界面。




<style>
    .terminal-span {
        margin: 0 0.1rem !important; 
        padding: 0.1rem 0.2rem !important; 
        background-color: #d3e3fd !important;
        color: black !important;
        border-radius: 0.25rem !important;
    }
    .terminal-div {
        padding: 1rem; 
        background-color: #d3e3fd; 
        color: black; 
        width: 40rem; 
        margin-bottom: 1rem; 
        border-radius: 0.25rem; 
        margin-left: 1rem; 
    }
</style>

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 发布前端服务

</div>

* #### 添加构建时环境变量（Build-time environment variables）

    <img src="../../Assets/react-dom.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>


* #### 创建Dockerfile

    <img src="../../Assets/react-dom.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>

* #### 提交并推送代码到Github

    <img src="../../Assets/tailwind-css-fix.jpg" alt="Social Insurance Query System" style="max-width: 80%; margin-left: 25px; padding: 50px; background-color: white">

    <div style="margin-bottom: 1rem"></div>

* #### 创建Cloud Run服务

  * 在Cursor的Chat(Agent模式)中输入: 
  
    * <span style="color: orange">
    
      _我的项目原型是在Google AI Studio上构建的，现在在本地构建时需要将TailWind以及其它依赖库的导入方式从运行时模式变更为构建时模式，请分析项目中的所有代码，并帮我完成修改。_

    </span>  

  <div style="margin-bottom: 1rem"></div>

* #### 在浏览器里进行测试

  * 打开<span class="terminal-span">Chrome浏览器</span>，并输入以下网址:   

    * _<span style="color: orange">
          http://localhost:3000
      </span>_

    * 我们将再次看到在第1章中所制作的原型界面，但是现在，它会从后端服务中获取数据，可以通过浏览器的<span class="terminal-span">DevTools->Network</span>页面来观察数据获取的过程。

    <div style="margin-bottom: 1rem"></div>
    
* #### 扩展知识
      
    * <span class="terminal-span">面向对象编程</span>（OOP）是最重要的编程思想和范式之一，类（Class）、对象（Object）、继承（Inherited）、多态（Polymorphism）、方法（Method）都是它的核心概念，深入理解面向对象的软件开发方法，是对一名软件工程师的基本要求。

    * <span class="terminal-span">Axios</span> 是一个基于 <span class="terminal-span">Promise</span> 的 HTTP 客户端，用于在浏览器和 Node.js 中发送网络请求。它能够自动转换 JSON 数据，提供更好的错误处理机制，同时还实现了<span class="terminal-span">拦截器</span>，这些特性极大提高了前端开发的效率。

    * <span class="terminal-span">DevTools</span>（Developer Tools）是直接内置在现代网络浏览器（如 Chrome, Firefox, Edge, Safari）中的一套强大的 Web 开发和调试工具集。在浏览器中按下<span class="terminal-span">Fn+F12</span>(MacOS)或<span class="terminal-span">F12</span>(Windows)即可调出DevTools的界面。




<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 作业

</div>

* 将前面章节中构建的前端和后端服务发布到Google Cloud Run上。

* 通过浏览器对算法进行验证，并与[Asatex工资计算器](https://revenue.asatex.jp/calculator)的计算结果进行比对。

<div style="border-left: 5px solid #007bff; padding-left: 15px;">  

## 作业交付方式  

</div>

  * Cloud Run App Url